In [48]:
# import packages used in this notebook
import os # need this to read the list of files in the directory
import io # need this to read the data from the zip file
from zipfile import ZipFile  # need this to read the data from the zip file
import tarfile # need this to read the data from the tar file
import json
import bz2
import gzip
import re # this is used for the regular expression to extract the subject from the file 

In [56]:
def loadtoMongo(contents, subject):
    from pymongo import MongoClient
    client = MongoClient('localhost', 27017)
    db = client['twitter']
    collection = db['tweets']
    subj = subject.lower()
    for line in contents:
        try:
            data = json.loads(line)
            # filter the data for my subject
            fltrData = re.search(r'\b{}\b'.format(subj), line.lower())

            if fltrData:
                try:
                    # if the subject is in the text, then load the entire line into mongo
                    content_dict = json.loads(line)
                    collection.insert_one(content_dict)
                except KeyError as e:
                    #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                    #print(line)
                    continue
        except json.decoder.JSONDecodeError as e:
            print("Error encountered at line: ", {e})
            continue
    
   

In [53]:
# tweetDrv = "/home/sba22230/shared-drives/E:/TwitterStream"
tweetDrv = "./SampleTweets/"
tweetWrappers = os.listdir(tweetDrv)
#display(tweetWrappers)


['2021',
 '20221101000000.json',
 '20221101235900.json',
 '29.json',
 'archive',
 'onetweet.txt',
 'SampleTweets.zip']

In [57]:
def bz2_func(data):
    data = bz2.decompress(data)
    data = io.StringIO(data.decode('utf-8'))
    contents = data.readlines()
    return contents

def gz_func(data):
    
    data = gzip.decompress(data)
    data = io.StringIO(data.decode('utf-8'))
    contents = data.readlines()
    return contents

for wrap in tweetWrappers:
    file = tweetDrv + wrap
    if file.endswith('.zip'):
        print("Zip file found") # need to a count of the files in the zip file format
        #display(file)
        with ZipFile(file, 'r') as zip:
            zipLn = len(zip.namelist())
            for i in range(zipLn):
                print(i, zip.namelist()[i])
                files = zip.namelist()[i]
                if files.endswith('bz2'):
                    data = zip.read(files)
                    contents = bz2_func(data) # now that we have the contents, we can push the tweets into mongoDB    
                    loadtoMongo(contents, 'euro')
                elif files.endswith('gz'):
                    data = zip.read(files)
                    contents = gz_func(data)
                    loadtoMongo(contents, 'euro')
                else:
                    print("unknown file" + os.path.splitext(files)[-1])
    elif file.endswith('.tar'):
        print("Tar file found") # need to a count of the files in the tar file format
        with tarfile.open(file, 'r') as tar:
            tarLn = len(tar.getnames())
            for i in range(tarLn):
                print(i, tar.getnames()[i])
                files = tar.getnames()[i]
                if files.endswith('bz2'):
                    data = tar.extractfile(files)
                    contents = bz2_func(data) # now that we have the contents, we can push the tweets into mongoDB    
                    loadtoMongo(contents, 'euro')
                elif files.endswith('gz'):
                    data = tar.extractfile(files)
                    contents = gz_func(data)
                    loadtoMongo(contents, 'euro')
    else:
        print("unknown file" + os.path.splitext(file)[-1])


unknown file
unknown file.json
unknown file.json
unknown file.json
unknown file
unknown file.txt
Zip file found
0 SampleTweets/2021/
unknown file
1 SampleTweets/2021/01/
unknown file
2 SampleTweets/2021/01/01/
unknown file
3 SampleTweets/2021/01/01/00/
unknown file
4 SampleTweets/2021/01/01/00/29.json.bz2
5 SampleTweets/20221101000000.json
unknown file.json
6 SampleTweets/20221101235900.json
unknown file.json
7 SampleTweets/29.json
unknown file.json
8 SampleTweets/archive/
unknown file
9 SampleTweets/archive/20221101000000.json.gz
10 SampleTweets/onetweet.txt
unknown file.txt
